
# Joining Cellartracker Metadata with Sample Tracker

The idea of this project is to prototype getting metadata for my wines, and for a list from the sample tracker table, inner join to the sample tracker table, then perform EDA on the join.

First, get the sample tracker table:

In [ ]:
import os

import pandas as pd

import numpy as np

os.getcwd()

In [ ]:
from google_sheets_api import get_sheets_values_as_df

def sample_tracker_df_builder():

    df = get_sheets_values_as_df(
        spreadsheet_id='15S2wm8t6ol2MRwTzgKTjlTcUgaStNlA22wJmFYhcwAY',
        range='sample_tracker!A1:H200',
        creds_parent_path=os.path.join(os.getcwd(),'credentials_tokens'),
        )
    return df

sample_tracker_df = sample_tracker_df_builder()

print(sample_tracker_df.columns)

sample_tracker_df = sample_tracker_df[['id','vintage', 'name', 'size', 'open_date', 'sample_date', 'variety', 'notes']]

import dtale

dtale.show(sample_tracker_df)

So we should be able to join on the name alone, through the use of a fuzzy match. But first, let's get the cellartracker table.

In [ ]:
from cellartracker import cellartracker

def get_cellar_tracker_table():

    client = cellartracker.CellarTracker('OctaneOolong', 'S74rg4z3r1')

    usecols = ['Size', 'Vintage', 'Wine', 'Locale', 'Country', 'Region', 'SubRegion', 'Appellation', 'Producer', 'Type', 'Color', 'Category', 'Varietal']

    cellar_tracker_df = pd.DataFrame(client.get_list())

    cellar_tracker_df = cellar_tracker_df[usecols]

    # clean it up. lower values and columns, replace 1001 with nv, check datatypes

    cellar_tracker_df = cellar_tracker_df.apply(lambda x : x.str.lower() if str(x) else x)
    cellar_tracker_df.columns = cellar_tracker_df.columns.str.lower()
    cellar_tracker_df = cellar_tracker_df.rename({'wine' : 'name'}, axis = 1)

    cellar_tracker_df = cellar_tracker_df.replace({'1001' : 'nv'})

    return cellar_tracker_df
    
cellar_tracker_df = get_cellar_tracker_table()

dtale.show(cellar_tracker_df)

That was easy! Now to join them. We should form a join col made up of the vintage and name in both tables.

According to chatgpt, "Pr&#333;ter&#333;" instead of "prōterō" is due to being encoded as a 'html character entity', and I can use the HTML module to 'unescape' the string.

In [ ]:
import html

def unescape_html(s):
    return html.unescape(s)

cellar_tracker_df = cellar_tracker_df.applymap(unescape_html)

dtale.show(cellar_tracker_df)

In [ ]:
def form_join_col(df):
    df['join_key'] = df['vintage'] + " " + df['name']
    return df

sample_tracker_df = form_join_col(sample_tracker_df)
cellar_tracker_df = form_join_col(cellar_tracker_df)
sample_tracker_df

In [ ]:
def join_dfs_with_fuzzy(df1, df2):
    from fuzzywuzzy import fuzz, process

    def fuzzy_match(s1, s2):
        return fuzz.token_set_ratio(s1, s2)
    
    df1['join_key_match'] = df1['join_key'].apply(lambda x: process.extractOne(x, df2['join_key'], scorer = fuzzy_match))

    # the above code produces a tuple of: ('matched_string', 'match score', 'matched_string_indice'). Usually it's two return values, but using scorer=fuzzy.token_sort_ratio or scorer=fuzz.token_set_ratio returns the index as well.

    df1['join_key_matched'] = df1['join_key_match'].apply(lambda x: x[0])
    df1['join_key_similarity'] = df1['join_key_match'].apply(lambda x : x[1])

    print(df1['join_key_similarity'][0])

    df1.drop(columns = ['join_key_match'], inplace = True)

    merged_df = pd.merge(df1, df2, left_on='join_key_matched', right_on='join_key')

    return merged_df

df = join_dfs_with_fuzzy(sample_tracker_df, cellar_tracker_df)

# original attempt fails on: 'null mystery', '2021 criante c21 cataratto', '2020 dom. simha paysan fleur sauvage', '2021 billaud d'or bourgogne blanc', 'null empty id, missing wine', '2021 st hugo gsm', '? yangarra 'roux beaute' roussanne'.

# I now want to have a look at all the incorrect matches and get theirx similarity scores. This should be simple enough, just need another column to keep the sim scores.

pd.options.display.min_rows = 50


dtale.show(df[['join_key_similarity','join_key_x', 'join_key_y']].sort_values('join_key_similarity'))

From this, a similarity score below 55 is when the matches became false. Thus a simple fix:

In [ ]:
def join_dfs_with_fuzzy(df1, df2):
    from fuzzywuzzy import fuzz, process

    def fuzzy_match(s1, s2):
        return fuzz.token_set_ratio(s1, s2)
    
    df1['join_key_match'] = df1['join_key'].apply(lambda x: process.extractOne(x, df2['join_key'], scorer = fuzzy_match))

    # the above code produces a tuple of: ('matched_string', 'match score', 'matched_string_indice'). Usually it's two return values, but using scorer=fuzzy.token_sort_ratio or scorer=fuzz.token_set_ratio returns the index as well.

    df1['join_key_matched'] = df1['join_key_match'].apply(lambda x: x[0] if x[1] > 65 else None)
    df1['join_key_similarity'] = df1['join_key_match'].apply(lambda x : x[1] if x[1] > 65 else None)

    df1.drop(columns = ['join_key_match'], inplace = True)

    merged_df = pd.merge(df1, df2, left_on='join_key_matched', right_on='join_key', how = 'left')

    return merged_df

df = join_dfs_with_fuzzy(sample_tracker_df, cellar_tracker_df)

dtale.show(df[['join_key_similarity','join_key_x', 'join_key_y']].sort_values('join_key_similarity'))

In [ ]:
def df_cleanup(df):

    df = df.drop(['vintage_x', 'name_x', 'join_key_x', 'join_key_matched', 'join_key_y', 'variety', 'size_y'], axis = 1)

    df = df.rename({
                    'size_x' : 'size',
                    'vintage_y' : 'vintage',
                    'name_y' : 'name'
                    }, axis = 1)
    
    ordered_cols = ['id', 'vintage', 'name', 'size']

    rest_of_cols = list(df.columns.drop(ordered_cols))

    ordered_cols = ordered_cols + rest_of_cols
    
    df = df[ordered_cols]
    
    display(df)

df_cleanup(df)

Ok looking good. Now to chuck it all into a google sheet.

Let's call the sheet..

sample_overview

In [ ]:
from pathlib import Path

sheet_url = 'https://docs.google.com/spreadsheets/d/15S2wm8t6ol2MRwTzgKTjlTcUgaStNlA22wJmFYhcwAY/edit#gid=347137817'
 

sheet_id = Path(sheet_url).parent.name

sheet_title = 'sample_overview'

creds_parent_path = Path.cwd() / 'credentials_tokens'

def make_result_sheet(sheet_id, sheet_title, creds_parent_path):

    from google_sheets_api import post_new_sheet

    sheet_title = 'sample_overview'

    creds_parent_path = Path.cwd() / 'credentials_tokens'

    response = post_new_sheet(sheet_id, sheet_title, creds_parent_path)

    display(response)

Now to post the results into the results sheet.

In [ ]:
def post_results(df, sheet_id, range, creds_parent_path):
    from google_sheets_api import post_df_as_sheet_values

    response = post_df_as_sheet_values(df, sheet_id, range, creds_parent_path)

    print(response)

#post_results(df, sheet_id, f"{sheet_title}!A1", creds_parent_path)

Once this is done, the next thing to do is to identify all samples which have been run on the avantor 10cm column. The most straightforward method would be to go back to your notes to check when you swapped over to it, since you haven't used another column since.

On checking, it appears that the 10cm column was installed on the 02/02. Now, from the table above, some of the wines only have a recorded open date, and some only have a recorded sample date.

To verify the sampling dates, we should go back to Agilette and see if we can form a table with the sample 'id' as the primary key, then join on that.

Regardless, looks like there are a lot of samples dated from after the 2nd. That being said, I think a run of samples before the 20's were all compromised by time before they were run. Definitely need to review the experiments.

That's enough in this note. Will continue in [the next notebook](./2023-03-28_tabulating-past-experiments.ipynb)